# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
import math

nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [64]:
import requests

url='https://norvig.com/big.txt'


try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    exit()

text_content = response.text

In [65]:
import re

sentences=[]
for i in text_content.split('.'):
    i = re.sub(r'[^a-zA-Z\s]', '', i).lower().replace('\n', '')
    tokens = word_tokenize(i)
    if tokens:
        sentences.append(tokens)

In [66]:
# Tokenize the text into words
words = [word for sentence in sentences for word in sentence if word]

# Build unigram, bigram, and trigram counts
unigram_counts = Counter(words)
bigram_counts = Counter(zip(words, words[1:]))
trigram_counts = Counter(zip(words, words[1:], words[2:]))

In [67]:
bigrams=''
with open('/kaggle/input/ngrams/bigrams.txt', 'r', encoding='utf-8', errors='replace') as f:
    bigrams=f.read().replace('\t', ' ').split('\n')

new_bi={}
words_new = []
for i in bigrams:
    num, w1, w2 = i.split()
    words_new.append(w1)
    words_new.append(w2)
    new_bi[(w1, w2)] = int(num)
    
bigram_counts = bigram_counts + Counter(new_bi)
unigram_counts = unigram_counts + Counter(words)

In [68]:
# from https://norvig.com/spell-correct.html
def edits1(word):
    """Generate all edits that are one edit away from `word`."""
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    """Generate all edits that are two edits away from `word`."""
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words):
    """Return the subset of `words` that appear in the dictionary."""
    return set(w for w in words if w in unigram_counts)

In [69]:
keyboard_layout = {
    'q': ['w', 'a'], 'w': ['q', 'e', 's'], 'e': ['w', 'r', 'd'], 'r': ['e', 't', 'f'],
    't': ['r', 'y', 'g'], 'y': ['t', 'u', 'h'], 'u': ['y', 'i', 'j'], 'i': ['u', 'o', 'k'],
    'o': ['i', 'p', 'l'], 'p': ['o'], 'a': ['q', 's', 'z'], 's': ['w', 'a', 'd', 'x'],
    'd': ['e', 's', 'f', 'c'], 'f': ['r', 'd', 'g', 'v'], 'g': ['t', 'f', 'h', 'b'],
    'h': ['y', 'g', 'j', 'n'], 'j': ['u', 'h', 'k', 'm'], 'k': ['i', 'j', 'l'],
    'l': ['o', 'k'], 'z': ['a', 'x'], 'x': ['s', 'z', 'c'], 'c': ['d', 'x', 'v'],
    'v': ['f', 'c', 'b'], 'b': ['g', 'v', 'n'], 'n': ['h', 'b', 'm'], 'm': ['j', 'n']
}

def keyboard_edits(word):
    """Generate edits based on keyboard layout."""
    edits = []
    for i in range(len(word)):
        for neighbor in keyboard_layout.get(word[i], []):
            edits.append(word[:i] + neighbor + word[i+1:])
    return set(edits)

In [88]:
def bigram_probability(bigram):
    """Calculate the probability of a bigram."""
    unigram = bigram[0]
    bigram_count = bigram_counts[bigram]
    unigram_count = unigram_counts[unigram]
    # print('From bigram\n',bigram, '\n', unigram, '\n')
    # print(bigram_count, unigram_count)
    return bigram_count / unigram_count if unigram_count > 0 else 0


def trigram_probability(trigram):
    """Calculate the probability of a trigram."""
    bigram = trigram[:2]
    trigram_count = trigram_counts[trigram]
    bigram_count = bigram_counts[bigram]
    # print('From trigram\n',trigram, '\n', bigram, '\n')
    # print(trigram_count, bigram_count)
    return trigram_count / bigram_count if bigram_count > 0 else 0


def candidates(word, prev_word, next_word):
    """Generate candidate corrections for `word` given its preceding word."""
    known_words = known([word]) or known(edits1(word)) or known(edits2(word)) or known(keyboard_edits(word))
    candidates = [(candidate, trigram_probability((prev_word, candidate, next_word))+bigram_probability((prev_word, candidate))) for candidate in known_words]
    # print(candidates)
    return sorted(candidates, key=lambda x: x[1], reverse=True)

In [76]:
def correct_sentence(sentence):
    """Correct spelling in a sentence."""
    corrected = []
    for i, word in enumerate(sentence):
        prev_word = sentence[i-1] if i > 0 else '<START>'
        next_word = sentence[i+1] if i < len(sentence)-1 else '<END>'
        candidates_list = candidates(word, prev_word, next_word)
        if candidates_list:
            best_candidate, _ = candidates_list[0]
            corrected.append(best_candidate)
        else:
            corrected.append(word)
    return corrected

In [89]:
sentences = ["i dking aport", 'dking species']
for sentence in sentences:
    tokens = word_tokenize(sentence.lower())
    corrected_tokens = correct_sentence(tokens)
    print( sentence, "-->", ' '.join(corrected_tokens))

i dking aport --> i doing sport
dking species --> dying species


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

---

#### **Choice of N-gram Dataset**

The data set was selected as the datasets "big.txt" from the Norvig website and dataset "bigrams.txt" from the useful data.

- **"bigrams.txt"** from the useful data contain ready-made bigrams that are convenient to use for unigrams and bigrams creation.

- **"big.txt"** from the Norvig website is suitable for creating unigrams, bigrams, and trigram creation

---

#### **Weights for Edit1, Edit2, and Absent Words Probabilities**

- **Edit1**: It makes correction that require one simple edit: deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter).

  The function edits1 returns a set of all the edited strings (whether words or not) that can be made with one simple edit.
  
- **Edit2**: It makes corrections that require two simple edits. This opens up a lot more possibilities, but usually few of them are known words.
  
    The function edits2 returns a set of all the edited strings (whether words or not) that can be made with two simple edits.

- **Absent Words**: If no valid correction exists in the dictionary, the original word was retained. This avoids introducing incorrect corrections for rare or proper nouns.

---

#### **Beam Search Parameters**
 The pruning mechanism combined with probability-based sorting achieves similar results to beam search without the additional complexity of maintaining a fixed beam width.

---

#### **Probability Calculation**
- **bigram probability**:  $P(w_t | w_{t-1})$ is calculated as:


  $
  P(w_t | w_{t-1}) = \frac{C(w_{t-1}, w_t)}{C(w_{t-1})}
  $
  
  where $ C(w_{t-1}, w_t) $ is the count of the bigram and $ C(w_{t-1}) $ is the count of the preceding word.

- **trigram probability**:  $ P(w_t | w_{t-2}, w_{t-1}) $ is calculated as:

  
  $
  P(w_t | w_{t-2}, w_{t-1}) = \frac{C(w_{t-2}, w_{t-1}, w_t)}{C(w_{t-2}, w_{t-1})}
  $

  where $ C(w_{t-2}, w_{t-1}, w_t) $ is the count of the trigram and $ C(w_{t-2}, w_{t-1}) $ is the count of the preceding bigram.

- **Combination of Bigram and Trigram Probabilities** : To incorporate wider contextual information, the final score for a candidate is the sum of its bigram and trigram probabilities:
  
  $
  \text{Score}(w_t) = P(w_t | w_{t-1}) + P(w_t | w_{t-2}, w_{t-1})
  $


---

#### **Keyboard Layout Awareness**

- Using the nearest letters of the keyboard layout increases the model's ability to cope with realistic typos.


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [149]:
import random
from nltk.corpus import words

nltk.download('words')
english_words = set(words.words())


def add_noise(word, error_prob=0.1):
    
    if random.random() > error_prob or len(word) <= 1:
        return word

    operation = random.choice(['delete', 'transpose', 'replace', 'insert'])
    idx = random.randint(0, len(word) - 1)

    if operation=='delete' and len(word)>1:
        return word[:idx]+word[idx+1:]
        
    elif operation == 'transpose' and idx < len(word) - 1:
        return word[:idx]+word[idx+1]+word[idx]+word[idx+2:]
        
    elif operation == 'replace':
        new_char = random.choice('abcdefghijklmnopqrstuvwxyz')
        return word[:idx]+new_char + word[idx+1:]
        
    elif operation=='insert':
        new_char = random.choice('abcdefghijklmnopqrstuvwxyz')
        return word[:idx]+new_char+word[idx:]
        
    return word

def generate_test_set(clean_sentences, error_prob=0.1):
    """Generate a noisy test set from clean sentences."""
    noisy_sentences = []
    for sentence in clean_sentences:
        noisy_sentence = [add_noise(word, error_prob) for word in sentence]
        noisy_sentences.append(noisy_sentence)
    return noisy_sentences


# Example
alices_intro = """
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: 
once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations 
in it, 'and what is the use of a book,' thought Alice 'without pictures or conversations?'
So she was considering, in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), 
whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when 
suddenly a White Rabbit with pink eyes ran close by her.'

"""

clean_sentences=[]
for i in alices_intro.split('.'):
    i = re.sub(r'[^a-zA-Z\s]', '', i).lower().replace('\n', '')
    tokens = word_tokenize(i)
    if tokens:
        clean_sentences.append(tokens)


noisy_sentences_low = generate_test_set(clean_sentences, 0.1)
noisy_sentences_medium = generate_test_set(clean_sentences, 0.5)
noisy_sentences_high = generate_test_set(clean_sentences, 1)

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [152]:
def evaluate_corrector(noisy_sentences, clean_sentences):
    """Evaluate the accuracy of a corrector on a test set."""
    total_words = 0
    correct_words = 0

    for noisy_sentence, clean_sentence in zip(noisy_sentences, clean_sentences):
        corrected_sentence = correct_sentence(noisy_sentence)
        for corrected_word, clean_word in zip(corrected_sentence, clean_sentence):
            if corrected_word == clean_word:
                correct_words += 1
            total_words += 1

    accuracy = correct_words / total_words if total_words > 0 else 0
    return accuracy


# Evaluate both correctors on the test sets
accuracy_context_sensitive_low = evaluate_corrector(noisy_sentences_low, clean_sentences)
accuracy_context_sensitive_medium = evaluate_corrector(noisy_sentences_medium, clean_sentences)
accuracy_context_sensitive_high = evaluate_corrector(noisy_sentences_high, clean_sentences)

print('Accuracy on 10% of incorrect text:', round(accuracy_context_sensitive_low, 2))
print('Accuracy on 50% of incorrect text:', round(accuracy_context_sensitive_medium, 2))
print('Accuracy on 100% of incorrect text:', round(accuracy_context_sensitive_high, 2))

Accuracy on 10% of incorrect text: 0.95
Accuracy on 50% of incorrect text: 0.74
Accuracy on 100% of incorrect text: 0.52


---

**Norwig's evaluator:**

---

In [111]:
def unit_tests():
    """
    Perform unit tests for the spelling corrector.
    """
    # Test individual word corrections
    assert correct_sentence(['speling']) == ['spelling']  # insert
    assert correct_sentence(['korrectud']) == ['corrected']  # replace 2
    assert correct_sentence(['bycycle']) == ['bicycle']  # replace
    assert correct_sentence(['inconvient']) == ['inconvenient']  # insert 2
    assert correct_sentence(['peotry']) == ['poetry']  # transpose
    assert correct_sentence(['peotryy']) == ['poetry']  # transpose + delete
    assert correct_sentence(['word']) == ['word']  # known
    assert correct_sentence(['quintessential']) == ['quintessential']  # unknown

    # Test sentence tokenization
    assert [word_tokenize('This is a TEST.') == ['this', 'is', 'a', 'test']]


    # Test probability calculations
    def P(word):
        return unigram_counts[word] / sum(unigram_counts.values())

    assert P('quintessential') == 0

    return "unit_tests pass"


# Run unit tests
print(unit_tests())

unit_tests pass


In [ ]:
def spelltest(tests, verbose=False):
    """
    Run correction on all (right, wrong) pairs in the test set and report results.
    """
    import time
    start = time.time()
    
    if not tests:  # Check if the test set is empty
        print("No test cases provided.")
        return
    
    good, unknown = 0, 0
    n = len(tests)

    for right, wrong in tests:
        corrected = correct_sentence([wrong])[0]  # Correct the word
        if corrected == right:
            good += 1
        else:
            if right not in unigram_counts:
                unknown += 1
            if verbose:
                print(f"correction({wrong}) => {corrected} (expected {right})")

    dt = time.time() - start
    print(f"{good / n:.2%} of {n} correct ({unknown / n:.2%} unknown) at {n / dt:.2f} words per second")


In [125]:
def Testset(lines):
    """
    Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs.
    """
    result = []
    lines = lines.split('\n')
    for line in lines:
        parts = line.strip().split(':')
        if len(parts) == 2:  # Ensure the line has the correct format
            right = parts[0].strip()
            wrongs = parts[1].strip().split()
            for wrong in wrongs:
                result.append((right, wrong))
    
    return result

In [92]:
import requests

url1='https://norvig.com/spell-testset1.txt'
url2='https://norvig.com/spell-testset2.txt'


try:
    response = requests.get(url1)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    exit()

text_content1 = response.text

try:
    response = requests.get(url2)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    exit()

text_content2 = response.text

In [126]:
spelltest(Testset(text_content1), verbose=False)  # Development set

spelltest(Testset(text_content2), verbose=False)  # Final test set

67.41% of 270 correct (6.30% unknown) at 31.31 words per second
61.75% of 400 correct (11.00% unknown) at 27.48 words per second


---

**Received  accuracies:**

---

##### My Solution of Evaluator:

Accuracy on 10% of incorrect text: 95%

Accuracy on 50% of incorrect text: 74%

Accuracy on 100% of incorrect text: 52%

##### The Norvig Corrector's Solution of Evaluator:

So on the development set we get 67.41% correct, and on the final test set we get 61.75% correct.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)